# Pytorch Lightning
#### Experiments using Pytorch Lightning to build custom CNN classification model.


Pytorch Lightning implementation
In this notebook I will implement CNN model using Pytorch Lightning. This model will be more flexible, than model from initial_experiments.ipynb, to provide more hyperparameters for training sessions.

In [3]:
from scripts.models.lightning_model import LightningCNN, ImageDataModule

In [4]:
import os
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()
root_data = os.getenv("KAGGLE_FILES_DIR")
dataset_path = Path(os.getcwd(), "..", root_data, 'processed')

Transformations by default are only resizing to 256x256.

In [5]:
datamodule = ImageDataModule(data_dir=dataset_path, batch_size=32)

# Training
1st model parameters:
- conv_layers: 5
- fc_layer_sizes: (256, 128)
- input_size: torch.Size([3, 256, 256])
- out_classes: 1
- initial_filters: 32
- hl_kernel_size: 5
- activation_func: nn.ReLU
- max_pool_kernel: 2
- dropout_conv: False
- dropout_fc: False
- dropout_rate: 0.5
- initial_learning_rate: 0.01
- loss_func: nn.BCEWithLogitsLoss
- optimizer: Adam
- metrics: Accuracy, Precision, Recall, F1, AUC, ConfusionMatrix

In [8]:
import torch
import pytorch_lightning as pl

model_1 = LightningCNN(
    conv_layers=5,
    fc_layer_sizes=(256, 128),
    input_size=torch.Size([3, 256, 256]),
    initial_filters=32,
    out_classes=1,
    hl_kernel_size=5,
    max_pool_kernel=2,
    dropout_conv=False,
    dropout_fc=False,
    initial_learning_rate=0.01
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',
    dirpath='../models/lightning',
    filename='model-9M-5conv-nodrop-{epoch:02d}-{valid_loss:.2f}',
    save_top_k=2,
    mode='min') 

early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',
    min_delta=0.001,
    patience=5,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, early_stopping]
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model_1, datamodule)


  | Name          | Type                  | Params
--------------------------------------------------------
0 | loss_func     | BCEWithLogitsLoss     | 0     
1 | accuracy      | BinaryAccuracy        | 0     
2 | precision     | BinaryPrecision       | 0     
3 | recall        | BinaryRecall          | 0     
4 | f1            | BinaryF1Score         | 0     
5 | auc           | BinaryAUROC           | 0     
6 | confmat       | BinaryConfusionMatrix | 0     
7 | hidden_layers | Sequential            | 4.4 M 
8 | fc_layers     | Sequential            | 4.8 M 
--------------------------------------------------------
9.1 M     Trainable params
0         Non-trainable params
9.1 M     Total params
36.429    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/isulim/Sages/retino-cnn/.venv/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric valid_loss did not improve in the last 5 records. Best score: 0.587. Signaling Trainer to stop.


In [11]:
trainer.test(model_1, datamodule)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7269971966743469     │
│         test_auc          │            0.0            │
│          test_f1          │            0.0            │
│         test_loss         │    0.5862252116203308     │
│      test_precision       │            0.0            │
│        test_recall        │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5862252116203308,
  'test_accuracy': 0.7269971966743469,
  'test_precision': 0.0,
  'test_recall': 0.0,
  'test_f1': 0.0,
  'test_auc': 0.0}]

Accuracy: 0.72 -> this is no better than distribution of classes in dataset.

# Training
2nd model parameters:
- conv_layers: 6
- fc_layer_sizes: (256, 128)
- input_size: torch.Size([3, 256, 256])
- out_classes: 1
- initial_filters: 32
- hl_kernel_size: 5
- activation_func: nn.ReLU
- max_pool_kernel: 2
- dropout_conv: True
- dropout_fc: True
- dropout_rate: 0.5
- initial_learning_rate: 0.01
- loss_func: nn.BCEWithLogitsLoss
- optimizer: Adam
- metrics: Accuracy, Precision, Recall, F1, AUC, ConfusionMatrix

In [14]:

model_2 = LightningCNN(
    conv_layers=6,
    fc_layer_sizes=(256, 128),
    input_size=torch.Size([3, 256, 256]),
    initial_filters=32,
    out_classes=1,
    hl_kernel_size=5,
    max_pool_kernel=2,
    dropout_conv=True,
    dropout_fc=True,
    dropout_rate=0.5,
    initial_learning_rate=0.01
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',
    dirpath='../models/lightning',
    filename='model-18M-6conv-0.5drop-{epoch:02d}-{valid_loss:.2f}',
    save_top_k=2,
    mode='min') 

early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',
    min_delta=0.001,
    patience=5,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, early_stopping]
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model_2, datamodule)


  | Name          | Type                  | Params
--------------------------------------------------------
0 | loss_func     | BCEWithLogitsLoss     | 0     
1 | accuracy      | BinaryAccuracy        | 0     
2 | precision     | BinaryPrecision       | 0     
3 | recall        | BinaryRecall          | 0     
4 | f1            | BinaryF1Score         | 0     
5 | auc           | BinaryAUROC           | 0     
6 | confmat       | BinaryConfusionMatrix | 0     
7 | hidden_layers | Sequential            | 17.5 M
8 | fc_layers     | Sequential            | 1.1 M 
--------------------------------------------------------
18.5 M    Trainable params
0         Non-trainable params
18.5 M    Total params
74.182    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/isulim/Sages/retino-cnn/.venv/lib/python3.12/site-packages/torchvision/__init__.py", line 6, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils
  File "/Users/isulim/Sages/retino-cnn/.venv/lib/python3.12/site-packages/torchvision/models/__init__.py", line 2, in <module>
    from .convnext import *
  File "/Users/isulim/Sages/retino-cnn/.venv/lib/python3.12/site-packages/torchvision/models/convnext.py", line 8, in <module>
    from 